## Profiling proseco / sparkles for impact of overlap prevention


In [1]:
import gzip
import pickle
import sys
from pathlib import Path

sys.path.insert(0, str(Path.home() / "git" / "proseco"))

In [2]:
import ska_helpers.utils

In [3]:
import proseco

print(proseco.test(get_version=True))
print(proseco.__file__)

5.12.1.dev5+g028bec8.d20240311-r941-028bec8
/Users/aldcroft/git/proseco/proseco/__init__.py


In [4]:
import sparkles

print(sparkles.test(get_version=True))
print(sparkles.__file__)

4.26.0
/Users/aldcroft/miniconda3/envs/ska3-flight-2024.1rc4/lib/python3.11/site-packages/sparkles/__init__.py


In [5]:
filename = "/Users/aldcroft/ska/data/mpcrit1/mplogs/2022/NOV2122/oflsa/output/NOV2122A_proseco.pkl.gz"
acas_dict = pickle.load(gzip.open(filename, "rb"))

In [6]:
def run_one(aca, run_proseco=True, run_sparkles=True, overlap_penalty=False):
    with ska_helpers.utils.temp_env_var(
        "PROSECO_DISABLE_OVERLAP_PENALTY", f"{not overlap_penalty}"
    ):
        if run_proseco:
            aca = proseco.get_aca_catalog(**aca.call_args)
        if run_sparkles:
            acar = aca.get_review_table()
            acar.run_aca_review()

In [7]:
def run_all(acas_dict, run_proseco=True, run_sparkles=True, overlap_penalty=False):
    for aca in acas_dict.values():
        run_one(
            aca,
            run_proseco=run_proseco,
            run_sparkles=run_sparkles,
            overlap_penalty=overlap_penalty,
        )

In [8]:
run_one(acas_dict[27565])

In [9]:
%prun -T tmp.txt -l 20 -s cumulative run_all(acas_dict, run_sparkles=False, overlap_penalty=False)

 
*** Profile printout saved to text file 'tmp.txt'.


         24042098 function calls (23194508 primitive calls) in 12.765 seconds

   Ordered by: cumulative time
   List reduced from 1409 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    133/1    0.000    0.000   12.766   12.766 {built-in method builtins.exec}
        1    0.008    0.008   12.766   12.766 1492753284.py:1(run_all)
       37    0.001    0.000   12.758    0.345 1317233785.py:1(run_one)
       37    0.000    0.000   12.757    0.345 __init__.py:6(get_aca_catalog)
       37    0.000    0.000   12.756    0.345 catalog.py:47(get_aca_catalog)
       37    0.004    0.000   12.756    0.345 catalog.py:82(_get_aca_catalog)
       37    0.004    0.000    7.766    0.210 acq.py:180(get_acq_catalog)
      709    0.080    0.000    5.133    0.007 acq.py:1553(__init__)
    14889    0.172    0.000    3.121    0.000 acq.py:880(calc_p_brightest)
       37    0.002    0.000    3.094    0.084 guide.py:33(get_guide_catalog)
       37    0.

In [10]:
%prun -T tmp.txt -l 30 -s cumulative run_all(acas_dict, run_sparkles=False, overlap_penalty=True)

 
*** Profile printout saved to text file 'tmp.txt'.


         24432417 function calls (23588458 primitive calls) in 12.983 seconds

   Ordered by: cumulative time
   List reduced from 1410 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    137/1    0.000    0.000   12.985   12.985 {built-in method builtins.exec}
        1    0.009    0.009   12.985   12.985 1492753284.py:1(run_all)
       37    0.000    0.000   12.976    0.351 1317233785.py:1(run_one)
       37    0.000    0.000   12.975    0.351 __init__.py:6(get_aca_catalog)
       37    0.000    0.000   12.975    0.351 catalog.py:47(get_aca_catalog)
       37    0.004    0.000   12.974    0.351 catalog.py:82(_get_aca_catalog)
       37    0.004    0.000    8.046    0.217 acq.py:180(get_acq_catalog)
      709    0.081    0.000    5.163    0.007 acq.py:1553(__init__)
    14889    0.173    0.000    3.142    0.000 acq.py:880(calc_p_brightest)
       37    0.002    0.000    3.104    0.084 guide.py:33(get_guide_catalog)
       37    0.

In [11]:
14.0 / 13.0

1.0769230769230769

In [12]:
%time run_one(acas_dict[27565])

CPU times: user 245 ms, sys: 3.24 ms, total: 248 ms
Wall time: 247 ms


In [13]:
acas_dict.keys()

dict_keys([27565.0, 27566.0, 25219.0, 27223.0, 27567.0, 27571.0, 26874.0, 27051.0, 26644.0, 45049.0, 45048.0, 45046.0, 45045.0, 27477.0, 25873.0, 25321.0, 27564.0, 27570.0, 27484.0, 26855.0, 26500.0, 45044.0, 45043.0, 45042.0, 45040.0, 45039.0, 45038.0, 26718.0, 27483.0, 25963.0, 27568.0, 27522.0, 25938.0, 27569.0, 25937.0, 26870.0, 45037.0])

In [14]:
%time run_all(acas_dict, run_sparkles=False, overlap_penalty=False)

CPU times: user 7.37 s, sys: 82.8 ms, total: 7.45 s
Wall time: 7.34 s


In [15]:
%time run_all(acas_dict, run_sparkles=False, overlap_penalty=True)

CPU times: user 7.45 s, sys: 81.6 ms, total: 7.53 s
Wall time: 7.41 s


In [16]:
%load_ext line_profiler

In [17]:
from proseco.acq import AcqTable

In [18]:
%lprun -f AcqTable.calc_p_safe run_all(acas_dict, run_sparkles=False, overlap_penalty=True)

Timer unit: 1e-09 s

Total time: 1.42739 s
File: /Users/aldcroft/git/proseco/proseco/acq.py
Function: calc_p_safe at line 978

Line #      Hits         Time  Per Hit   % Time  Line Contents
   978                                               def calc_p_safe(self, verbose=False):
   979                                                   """
   980                                                   Calculate the probability of a safing action resulting from failure
   981                                                   to acquire at least two (2) acquisition stars.
   982                                           
   983                                                   This uses the probability of 2 or fewer stars => "conservative" p_fail at this
   984                                                   man_err.  This uses 2 stars instead of 1 or fewer (which is the actual criteria
   985                                                   for a safing action).  This allows for one star t

In [19]:
%lprun -f AcqTable.get_overlap_penalties run_all(acas_dict, run_sparkles=False, overlap_penalty=True)

Timer unit: 1e-09 s

Total time: 0.42878 s
File: /Users/aldcroft/git/proseco/proseco/acq.py
Function: get_overlap_penalties at line 938

Line #      Hits         Time  Per Hit   % Time  Line Contents
   938                                               def get_overlap_penalties(self):
   939                                                   """
   940                                                   Get the penalties for overlapping boxes.
   941                                           
   942                                                   :returns: list of penalties (float)
   943                                                   """
   944      5678    3679000.0    647.9      0.9          n_acq = len(self)
   945      5678   13418000.0   2363.2      3.1          penalties = np.ones(n_acq)
   946      5678   11273000.0   1985.4      2.6          if os.environ.get("PROSECO_DISABLE_OVERLAP_PENALTY") == "True":
   947                                                       return pen